In [107]:
import pandas as pd
import numpy as np
import os
import sys
# My original function
from ImportFunctions import *

input_path = "../input/"
out_path = "../output/"
out_file = "accused.csv"

files = [input_path + f for f in os.listdir(input_path)]

data = pd.DataFrame()
metadata = pd.DataFrame()

for f in files:
    df = ReadMessy(f)
    df.insert(0, "CRID", (df["Number:"]
                            .fillna(method='ffill')
                            .astype(int)))
    df = (df.drop("Number:", axis=1)
            .dropna(thresh = 2))
    df.columns = ["CRID", "Full.Name", "Birth.Year", "Gender", "Race", "Appointed.Date", "Current.Unit", "Current.Rank", "Star","Complaint.Category", "Recommended.Finding", "Recommended.Discipline", "Final.Finding", "Final.Discipline"]
    
    data = (data
            .append(df)
            .reset_index(drop=True))

    metadata = (metadata
                    .append(metadata_dataset(df, f, out_file)
                    .reset_index(drop=True)))
print(len(data['Full.Name'].unique()))
print(data.shape)

15494
(125581, 14)


In [82]:
def ReadMessy(path, add_skip=1):
    df = pd.read_excel(path, rows=20)
    col_list = df.columns.tolist()
    # if [col if '\n' in col or len(col) > 40 for col in col_list]:
    #   metadata = True
    Report_Produced_Date = [x for x in col_list if isinstance(x, datetime.datetime)]
    col_list = [x for x in col_list if x not in Report_Produced_Date]
    FOIA_Request = [x for x in col_list if 'FOIA' in x][0]

    skip = np.where(df.iloc[:,0].str.contains('Number', na=False))[0][0]+add_skip
    df = (pd.read_excel(path, skiprows=skip)
            .dropna(how='all', axis=0)
            .dropna(how='all', axis=1))

    return df

In [37]:
add_skip = 1
row_sums = 0
for path in files:
    df = pd.read_excel(path, rows=20)
    col_list = df.columns.tolist()
    # if [col if '\n' in col or len(col) > 40 for col in col_list]:
    #   metadata = True
    Report_Produced_Date = [x for x in col_list if isinstance(x, datetime.datetime)]
    col_list = [x for x in col_list if x not in Report_Produced_Date]
    FOIA_Request = [x for x in col_list if 'FOIA' in x][0]

    skip = np.where(df.iloc[:,0].str.contains('Number', na=False))[0][0]+add_skip
    df = (pd.read_excel(path, skiprows=skip)
            .dropna(how='all', axis=0)
            .dropna(how='all', axis=1))
    df.insert(0, "CRID", (df["Number:"]
                            .fillna(method='ffill')
                            .astype(int)))
    df = df.drop("Number:", axis=1)
    df = df.dropna(thresh = len(df.columns.values)-1)
    row_sums += df.shape[0]


In [38]:
row_sums

84970

In [40]:
path = files[1]
df = pd.read_excel(path, rows=20)
col_list = df.columns.tolist()
Report_Produced_Date = [x for x in col_list if isinstance(x, datetime.datetime)]
col_list = [x for x in col_list if x not in Report_Produced_Date]
FOIA_Request = [x for x in col_list if 'FOIA' in x][0]
skip = np.where(df.iloc[:,0].str.contains('Number', na=False))[0][0]+add_skip
print(skip)

10


In [59]:
df = pd.read_excel(path, skiprows=skip)
print(df.shape)
df = df.dropna(how='all', axis=0).dropna(how='all', axis=1)
print(df.shape)
df.insert(0, "CRID", (df["Number:"]
                            .fillna(method='ffill')
                            .astype(int)))
df = df.drop("Number:", axis=1)
print(df.shape)
df = df.dropna(thresh = 1)
print(df.shape)

(28064, 16)
(28064, 14)
(28064, 14)
(17414, 14)


In [102]:
# Tony's Code
final_df = pd.DataFrame()
def null_dropper(df):
    buf = io.StringIO()
    df.info(buf=buf)
    s = buf.getvalue()
    info_values = [re.split("\\s\\s+",x) for x in s.split("\n")]
    info_values = [x for x in info_values if len(x)>1]
    info_values = [x[0] for x in info_values if x[1].startswith('0 non-null')]
    df = df.drop(info_values,axis=1)
    return df

for file in files:
    #df = pd.read_excel(in_path_nov_2016_report + file,nrows=20)
    df = pd.read_excel(file, skip_rows=20)
    ## Making Sure Every File contains date the file was created and the foia that created it
    col_list = df.columns.tolist()
    Report_Produced_Date = [x for x in col_list if isinstance(x, datetime.datetime)]
    col_list = [x for x in col_list if isinstance(x, datetime.datetime)==False]
    FOIA_Request = [x for x in col_list if 'FOIA' in x][0]
    # +1 because of python indexing,
    skip = np.where(df.iloc[:,0]=="Number:")[0][0]+1
    #df = pd.read_excel(in_path_nov_2016_report + file, skiprows=skip)
    df = pd.read_excel(file, skiprows=skip)
    print(df.shape)
    print(len(df['Accused:'].unique()))
    df.dropna(how='all', inplace=True)
    ## Remove leading and trailing whitespace from columns 
    df.columns = [col.strip() if col not in Report_Produced_Date else col for col in df.columns.tolist()]
    ## Need to fill in Number
    df['Number:'].fillna(method='ffill', inplace=True)
    df['Number:'] = df['Number:'].astype(int)

    ## Drops end of record 
    #print(df.head())
    print(df.shape)
    print(len(df['Accused:'].unique()))
    df = df.dropna(thresh=2)
    #df = df.dropna(subset=["Accused:","Gender:","Date of Appt:","Star:"],how="all",axis=0)
    #print(df.head())
    print(df.shape)
    print(len(df['Accused:'].unique()))


    ## drops the significant number of columns that are all nulls
    #df = null_dropper(df)
    df = df.dropna(how='all', axis=1)
    print(df.shape)
    print(len(df['Accused:'].unique()))


    ##print(df.head())
    df.columns = ["CRID","Accused","Accused_Birth_Year","Accused_Gender","Accused_Race_Code","Date_of_Appt","Current_Unit","Current_Rank",
                  "Star","Complaint_Category","Orig_Finding","Orig_Recommended_Discipline","Final_Finding", "Final_Recommended_Discipline"]
    final_df = final_df.append(df)
    final_df.reset_index(drop=True,inplace=True)
    print("*****")
print(final_df.shape)
print(len(final_df['Accused'].unique()))

(52106, 16)
9376
(52106, 16)
9376
(33173, 16)
9375
(33173, 14)
9375
*****
(28064, 16)
7045
(28064, 16)
7045
(18202, 16)
7044
(18202, 14)
7044
*****
(35083, 16)
8261
(35083, 16)
8261
(23153, 16)
8260
(23153, 14)
8260
*****
(45525, 16)
8724
(45525, 16)
8724
(30209, 16)
8723
(30209, 14)
8723
*****
(32249, 16)
7793
(32249, 16)
7793
(20844, 16)
7792
(20844, 14)
7792
*****
(125581, 14)
15494
